In [1]:
import re
import csv
import nltk
import keras
import string
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from nltk import FreqDist
from termcolor import colored
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from mlxtend.plotting import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score
from tensorflow.keras.layers import LSTM, Dense, Dropout, GlobalMaxPooling1D, Bidirectional, Embedding

#Classifiers
from sklearn.naive_bayes import MultinomialNB

## Importing Dataset

In [2]:
train_set_path = '/kaggle/input/ag-news-sinhala/ag_news_sinhala/train.csv'
test_set_path = '/kaggle/input/ag-news-sinhala/ag_news_sinhala/test.csv'

In [3]:
stop_words = '/kaggle/input/stop-words/stop words.txt'
stem_dictionary = '/kaggle/input/stem-dictionary/stem_dictionary.txt'

In [4]:
train_set = pd.read_csv(train_set_path)
test_set = pd.read_csv(test_set_path)

In [5]:
train_set.head()

,Unnamed: 0,Class Index,Title (English),Description (English),Title (Sinhala),Description (Sinhala)
0,0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය.,"නිව් යෝර්ක් (රොයිටර්) - කෙටි විකුණුම්කරුවන්, ව..."
1,1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්),රොයිටර් - පුද්ගලික ආයෝජන සමාගමක් වන Carlyle Gr...
2,2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,තෙල් සහ ආර්ථිකය,රොයිටර් - බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබ...
3,3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,රොයිටර් - කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම්වල...
4,4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...",එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"AFP - ඉරාක ලෝක තෙල් මිල, වාර්තා පෙරලීම සහ මුදල..."


In [6]:
test_set.head()

,Unnamed: 0,Class Index,Title (English),Description (English),Title (Sinhala),Description (Sinhala)
0,0,3,Fears for T N pension after talks,Unions representing workers at Turner Newall...,කතා කරලා TN විශ්‍රාම වැටුපට බයයි,Turner Newall හි කම්කරුවන් නියෝජනය කරන වෘත්තීය...
1,1,4,The Race is On: Second Private Team Sets Launc...,A second team of rocketeers competing for the ...,තරඟය ක්‍රියාත්මකයි: දෙවන පුද්ගලික කණ්ඩායම මානව...,මිලියනයක් වන අන්සාරි X ත්‍යාගය සඳහා තරඟ වදින ද...
2,2,4,Ky. Company Wins Grant to Study Peptides (AP),A company founded by a chemistry researcher at...,K. සමාගම Peptides (AP) අධ්‍යයනය සඳහා ප්‍රදානයක...,ලුයිස්විල් විශ්ව විද්‍යාලයේ රසායන විද්‍යා පර්ය...
3,3,4,Prediction Unit Helps Forecast Wildfires (AP),It's barely dawn when Mike Fitzpatrick starts ...,පුරෝකථන ඒකකය ලැව්ගිනි (AP) පුරෝකථනය කිරීමට උපක...,"මයික් ෆිට්ස්පැට්‍රික් වර්ණවත් සිතියම්, රූප සහ ..."
4,4,4,Calif. Aims to Limit Farm-Related Smog (AP),Southern California's smog-fighting agency wen...,කැලිෆෝනියාව. ගොවිපල ආශ්‍රිත දුමාරය (AP) සීමා ක...,දකුණු කැලිෆෝනියාවේ දුමාරයට එරෙහිව සටන් කිරීමේ ...


In [7]:
train_set = train_set.drop(['Unnamed: 0', 'Title (English)', 'Description (English)'], axis=1).copy()

In [8]:
train_set.head()

,Class Index,Title (Sinhala),Description (Sinhala)
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය.,"නිව් යෝර්ක් (රොයිටර්) - කෙටි විකුණුම්කරුවන්, ව..."
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්),රොයිටර් - පුද්ගලික ආයෝජන සමාගමක් වන Carlyle Gr...
2,3,තෙල් සහ ආර්ථිකය,රොයිටර් - බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබ...
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,රොයිටර් - කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම්වල...
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"AFP - ඉරාක ලෝක තෙල් මිල, වාර්තා පෙරලීම සහ මුදල..."


In [9]:
test_set = test_set.drop(['Unnamed: 0', 'Title (English)', 'Description (English)'], axis=1).copy()

In [10]:
test_set.head()

,Class Index,Title (Sinhala),Description (Sinhala)
0,3,කතා කරලා TN විශ්‍රාම වැටුපට බයයි,Turner Newall හි කම්කරුවන් නියෝජනය කරන වෘත්තීය...
1,4,තරඟය ක්‍රියාත්මකයි: දෙවන පුද්ගලික කණ්ඩායම මානව...,මිලියනයක් වන අන්සාරි X ත්‍යාගය සඳහා තරඟ වදින ද...
2,4,K. සමාගම Peptides (AP) අධ්‍යයනය සඳහා ප්‍රදානයක...,ලුයිස්විල් විශ්ව විද්‍යාලයේ රසායන විද්‍යා පර්ය...
3,4,පුරෝකථන ඒකකය ලැව්ගිනි (AP) පුරෝකථනය කිරීමට උපක...,"මයික් ෆිට්ස්පැට්‍රික් වර්ණවත් සිතියම්, රූප සහ ..."
4,4,කැලිෆෝනියාව. ගොවිපල ආශ්‍රිත දුමාරය (AP) සීමා ක...,දකුණු කැලිෆෝනියාවේ දුමාරයට එරෙහිව සටන් කිරීමේ ...


## Labels

In [11]:
labels = ['World', 'Sports', 'Business & Economics', 'Science & Technology']

In [12]:
def clean_text(text):
    tokens = nltk.word_tokenize(text)
    regex = re.compile(u'[^\u0D80-\u0DFF]', re.UNICODE)
    tokens = [regex.sub('', w) for w in tokens]
    tokens = [c for c in tokens if c not in string.punctuation]
    return tokens

In [13]:
test_set['Cleaned Title'] = test_set['Title (Sinhala)'].apply(lambda x: clean_text(x))
test_set.head()

,Class Index,Title (Sinhala),Description (Sinhala),Cleaned Title
0,3,කතා කරලා TN විශ්‍රාම වැටුපට බයයි,Turner Newall හි කම්කරුවන් නියෝජනය කරන වෘත්තීය...,"[කතා, කරලා, විශ්රාම, වැටුපට, බයයි]"
1,4,තරඟය ක්‍රියාත්මකයි: දෙවන පුද්ගලික කණ්ඩායම මානව...,මිලියනයක් වන අන්සාරි X ත්‍යාගය සඳහා තරඟ වදින ද...,"[තරඟය, ක්රියාත්මකයි, දෙවන, පුද්ගලික, කණ්ඩායම, ..."
2,4,K. සමාගම Peptides (AP) අධ්‍යයනය සඳහා ප්‍රදානයක...,ලුයිස්විල් විශ්ව විද්‍යාලයේ රසායන විද්‍යා පර්ය...,"[සමාගම, අධ්යයනය, සඳහා, ප්රදානයක්, දිනා, ගනී]"
3,4,පුරෝකථන ඒකකය ලැව්ගිනි (AP) පුරෝකථනය කිරීමට උපක...,"මයික් ෆිට්ස්පැට්‍රික් වර්ණවත් සිතියම්, රූප සහ ...","[පුරෝකථන, ඒකකය, ලැව්ගිනි, පුරෝකථනය, කිරීමට, උප..."
4,4,කැලිෆෝනියාව. ගොවිපල ආශ්‍රිත දුමාරය (AP) සීමා ක...,දකුණු කැලිෆෝනියාවේ දුමාරයට එරෙහිව සටන් කිරීමේ ...,"[කැලිෆෝනියාව, ගොවිපල, ආශ්රිත, දුමාරය, සීමා, කි..."


In [14]:
test_set['Cleaned Description'] = test_set['Description (Sinhala)'].apply(lambda x: clean_text(x))
test_set.head()

,Class Index,Title (Sinhala),Description (Sinhala),Cleaned Title,Cleaned Description
0,3,කතා කරලා TN විශ්‍රාම වැටුපට බයයි,Turner Newall හි කම්කරුවන් නියෝජනය කරන වෘත්තීය...,"[කතා, කරලා, විශ්රාම, වැටුපට, බයයි]","[හි, කම්කරුවන්, නියෝජනය, කරන, වෘත්තීය, සමිති, ..."
1,4,තරඟය ක්‍රියාත්මකයි: දෙවන පුද්ගලික කණ්ඩායම මානව...,මිලියනයක් වන අන්සාරි X ත්‍යාගය සඳහා තරඟ වදින ද...,"[තරඟය, ක්රියාත්මකයි, දෙවන, පුද්ගලික, කණ්ඩායම, ...","[මිලියනයක්, වන, අන්සාරි, ත්යාගය, සඳහා, තරඟ, වද..."
2,4,K. සමාගම Peptides (AP) අධ්‍යයනය සඳහා ප්‍රදානයක...,ලුයිස්විල් විශ්ව විද්‍යාලයේ රසායන විද්‍යා පර්ය...,"[සමාගම, අධ්යයනය, සඳහා, ප්රදානයක්, දිනා, ගනී]","[ලුයිස්විල්, විශ්ව, විද්යාලයේ, රසායන, විද්යා, ..."
3,4,පුරෝකථන ඒකකය ලැව්ගිනි (AP) පුරෝකථනය කිරීමට උපක...,"මයික් ෆිට්ස්පැට්‍රික් වර්ණවත් සිතියම්, රූප සහ ...","[පුරෝකථන, ඒකකය, ලැව්ගිනි, පුරෝකථනය, කිරීමට, උප...","[මයික්, ෆිට්ස්පැට්රික්, වර්ණවත්, සිතියම්, රූප,..."
4,4,කැලිෆෝනියාව. ගොවිපල ආශ්‍රිත දුමාරය (AP) සීමා ක...,දකුණු කැලිෆෝනියාවේ දුමාරයට එරෙහිව සටන් කිරීමේ ...,"[කැලිෆෝනියාව, ගොවිපල, ආශ්රිත, දුමාරය, සීමා, කි...","[දකුණු, කැලිෆෝනියාවේ, දුමාරයට, එරෙහිව, සටන්, ක..."


In [15]:
train_set['Cleaned Title'] = train_set['Title (Sinhala)'].apply(lambda x: clean_text(x))
train_set.head()

,Class Index,Title (Sinhala),Description (Sinhala),Cleaned Title
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය.,"නිව් යෝර්ක් (රොයිටර්) - කෙටි විකුණුම්කරුවන්, ව...","[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,..."
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්),රොයිටර් - පුද්ගලික ආයෝජන සමාගමක් වන Carlyle Gr...,"[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]"
2,3,තෙල් සහ ආර්ථිකය,රොයිටර් - බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබ...,"[තෙල්, සහ, ආර්ථිකය]"
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,රොයිටර් - කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම්වල...,"[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,..."
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"AFP - ඉරාක ලෝක තෙල් මිල, වාර්තා පෙරලීම සහ මුදල...","[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක..."


In [16]:
train_set['Cleaned Description'] = train_set['Description (Sinhala)'].apply(lambda x: clean_text(x))
train_set.head()

,Class Index,Title (Sinhala),Description (Sinhala),Cleaned Title,Cleaned Description
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය.,"නිව් යෝර්ක් (රොයිටර්) - කෙටි විකුණුම්කරුවන්, ව...","[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,...","[නිව්, යෝර්ක්, රොයිටර්, කෙටි, විකුණුම්කරුවන්, ..."
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්),රොයිටර් - පුද්ගලික ආයෝජන සමාගමක් වන Carlyle Gr...,"[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]","[රොයිටර්, පුද්ගලික, ආයෝජන, සමාගමක්, වන, ආරක්ෂක..."
2,3,තෙල් සහ ආර්ථිකය,රොයිටර් - බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබ...,"[තෙල්, සහ, ආර්ථිකය]","[රොයිටර්, බොරතෙල්, මිල, ඉහළ, යාම, සහ, ආර්ථිකය,..."
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,රොයිටර් - කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම්වල...,"[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,...","[රොයිටර්, කැරලිකාර, මිලීෂියාවකට, යටිතල, පහසුකම..."
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"AFP - ඉරාක ලෝක තෙල් මිල, වාර්තා පෙරලීම සහ මුදල...","[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක...","[ඉරාක, ලෝක, තෙල්, මිල, වාර්තා, පෙරලීම, සහ, මුද..."


In [17]:
stopwordsreader = open('/kaggle/input/stop-words/stop words.txt', 'r', encoding='utf-8')
reader = csv.reader(stopwordsreader)

swords = []
for row in reader:
    swords.append(row[0])
print(swords)

['සහ', 'සමග', 'සමඟ', 'අහා', 'ආහ්', 'ආ', 'ඕහෝ', 'අනේ', 'අඳෝ', 'අපොයි', 'අපෝ', 'අයියෝ', 'ආයි', 'ඌයි', 'චී', 'චිහ්', 'චික්', 'හෝ\u200d', 'දෝ', 'දෝහෝ', 'මෙන්', 'සේ', 'වැනි', 'බඳු', 'වන්', 'අයුරු', 'අයුරින්', 'ලෙස', 'වැඩි', 'ශ්\u200dරී', 'හා', 'ය', 'නිසා', 'නිසාවෙන්', 'බවට', 'බව', 'බවෙන්', 'නම්', 'වැඩි', 'සිට', 'දී', 'මහා', 'මහ', 'පමණ', 'පමණින්', 'පමන', 'වන', 'විට', 'විටින්', 'මේ', 'මෙලෙස', 'මෙයින්', 'ඇති', 'ලෙස', 'සිදු', 'වශයෙන්', 'යන', 'සඳහා', 'මගින්', 'හෝ\u200d', 'ඉතා', 'ඒ', 'එම', 'ද', 'අතර', 'විසින්', 'සමග', 'පිළිබඳව', 'පිළිබඳ', 'තුළ', 'බව', 'වැනි', 'මහ', 'මෙම', 'මෙහි', 'මේ', 'වෙත', 'වෙතින්', 'වෙතට', 'වෙනුවෙන්', 'වෙනුවට', 'වෙන', 'ගැන', 'නෑ', 'අනුව', 'නව', 'පිළිබඳ', 'විශේෂ', 'දැනට', 'එහෙන්', 'මෙහෙන්', 'එහේ', 'මෙහේ', 'ම', 'තවත්', 'තව ', 'සහ', 'දක්වා', 'ට', 'ගේ', 'එ', 'ක', 'ක්', 'බවත්', 'බවද', 'මත', 'ඇතුලු', 'ඇතුළු', 'මෙසේ', 'වඩා', 'වඩාත්ම', 'නිති', 'නිතිත්', 'නිතොර', 'නිතර', 'ඉක්බිති', 'දැන්', 'යලි', 'පුන', 'ඉතින්', 'සිට', 'සිටන්', 'පටන්', 'තෙක්', 'දක්වා', 'සා', 'තාක්', 'තුවක්', 'පවා', 'ද

In [18]:
def remove_stop_words(tokens):
    regex = re.compile(u'[^\u0D80-\u0DFF]', re.UNICODE)
    tokens = [regex.sub('', w) for w in tokens]
    tokens = [c for c in tokens if c not in string.punctuation]
    return tokens

In [19]:
test_set['Stopwords Removed (Title)'] = test_set['Cleaned Title'].apply(lambda x: remove_stop_words(x))
test_set.head()

,Class Index,Title (Sinhala),Description (Sinhala),Cleaned Title,Cleaned Description,Stopwords Removed (Title)
0,3,කතා කරලා TN විශ්‍රාම වැටුපට බයයි,Turner Newall හි කම්කරුවන් නියෝජනය කරන වෘත්තීය...,"[කතා, කරලා, විශ්රාම, වැටුපට, බයයි]","[හි, කම්කරුවන්, නියෝජනය, කරන, වෘත්තීය, සමිති, ...","[කතා, කරලා, විශ්රාම, වැටුපට, බයයි]"
1,4,තරඟය ක්‍රියාත්මකයි: දෙවන පුද්ගලික කණ්ඩායම මානව...,මිලියනයක් වන අන්සාරි X ත්‍යාගය සඳහා තරඟ වදින ද...,"[තරඟය, ක්රියාත්මකයි, දෙවන, පුද්ගලික, කණ්ඩායම, ...","[මිලියනයක්, වන, අන්සාරි, ත්යාගය, සඳහා, තරඟ, වද...","[තරඟය, ක්රියාත්මකයි, දෙවන, පුද්ගලික, කණ්ඩායම, ..."
2,4,K. සමාගම Peptides (AP) අධ්‍යයනය සඳහා ප්‍රදානයක...,ලුයිස්විල් විශ්ව විද්‍යාලයේ රසායන විද්‍යා පර්ය...,"[සමාගම, අධ්යයනය, සඳහා, ප්රදානයක්, දිනා, ගනී]","[ලුයිස්විල්, විශ්ව, විද්යාලයේ, රසායන, විද්යා, ...","[සමාගම, අධ්යයනය, සඳහා, ප්රදානයක්, දිනා, ගනී]"
3,4,පුරෝකථන ඒකකය ලැව්ගිනි (AP) පුරෝකථනය කිරීමට උපක...,"මයික් ෆිට්ස්පැට්‍රික් වර්ණවත් සිතියම්, රූප සහ ...","[පුරෝකථන, ඒකකය, ලැව්ගිනි, පුරෝකථනය, කිරීමට, උප...","[මයික්, ෆිට්ස්පැට්රික්, වර්ණවත්, සිතියම්, රූප,...","[පුරෝකථන, ඒකකය, ලැව්ගිනි, පුරෝකථනය, කිරීමට, උප..."
4,4,කැලිෆෝනියාව. ගොවිපල ආශ්‍රිත දුමාරය (AP) සීමා ක...,දකුණු කැලිෆෝනියාවේ දුමාරයට එරෙහිව සටන් කිරීමේ ...,"[කැලිෆෝනියාව, ගොවිපල, ආශ්රිත, දුමාරය, සීමා, කි...","[දකුණු, කැලිෆෝනියාවේ, දුමාරයට, එරෙහිව, සටන්, ක...","[කැලිෆෝනියාව, ගොවිපල, ආශ්රිත, දුමාරය, සීමා, කි..."


In [20]:
test_set['Stopwords Removed (Description)'] = test_set['Cleaned Description'].apply(lambda x: remove_stop_words(x))
test_set.head()

,Class Index,Title (Sinhala),Description (Sinhala),Cleaned Title,Cleaned Description,Stopwords Removed (Title),Stopwords Removed (Description)
0,3,කතා කරලා TN විශ්‍රාම වැටුපට බයයි,Turner Newall හි කම්කරුවන් නියෝජනය කරන වෘත්තීය...,"[කතා, කරලා, විශ්රාම, වැටුපට, බයයි]","[හි, කම්කරුවන්, නියෝජනය, කරන, වෘත්තීය, සමිති, ...","[කතා, කරලා, විශ්රාම, වැටුපට, බයයි]","[හි, කම්කරුවන්, නියෝජනය, කරන, වෘත්තීය, සමිති, ..."
1,4,තරඟය ක්‍රියාත්මකයි: දෙවන පුද්ගලික කණ්ඩායම මානව...,මිලියනයක් වන අන්සාරි X ත්‍යාගය සඳහා තරඟ වදින ද...,"[තරඟය, ක්රියාත්මකයි, දෙවන, පුද්ගලික, කණ්ඩායම, ...","[මිලියනයක්, වන, අන්සාරි, ත්යාගය, සඳහා, තරඟ, වද...","[තරඟය, ක්රියාත්මකයි, දෙවන, පුද්ගලික, කණ්ඩායම, ...","[මිලියනයක්, වන, අන්සාරි, ත්යාගය, සඳහා, තරඟ, වද..."
2,4,K. සමාගම Peptides (AP) අධ්‍යයනය සඳහා ප්‍රදානයක...,ලුයිස්විල් විශ්ව විද්‍යාලයේ රසායන විද්‍යා පර්ය...,"[සමාගම, අධ්යයනය, සඳහා, ප්රදානයක්, දිනා, ගනී]","[ලුයිස්විල්, විශ්ව, විද්යාලයේ, රසායන, විද්යා, ...","[සමාගම, අධ්යයනය, සඳහා, ප්රදානයක්, දිනා, ගනී]","[ලුයිස්විල්, විශ්ව, විද්යාලයේ, රසායන, විද්යා, ..."
3,4,පුරෝකථන ඒකකය ලැව්ගිනි (AP) පුරෝකථනය කිරීමට උපක...,"මයික් ෆිට්ස්පැට්‍රික් වර්ණවත් සිතියම්, රූප සහ ...","[පුරෝකථන, ඒකකය, ලැව්ගිනි, පුරෝකථනය, කිරීමට, උප...","[මයික්, ෆිට්ස්පැට්රික්, වර්ණවත්, සිතියම්, රූප,...","[පුරෝකථන, ඒකකය, ලැව්ගිනි, පුරෝකථනය, කිරීමට, උප...","[මයික්, ෆිට්ස්පැට්රික්, වර්ණවත්, සිතියම්, රූප,..."
4,4,කැලිෆෝනියාව. ගොවිපල ආශ්‍රිත දුමාරය (AP) සීමා ක...,දකුණු කැලිෆෝනියාවේ දුමාරයට එරෙහිව සටන් කිරීමේ ...,"[කැලිෆෝනියාව, ගොවිපල, ආශ්රිත, දුමාරය, සීමා, කි...","[දකුණු, කැලිෆෝනියාවේ, දුමාරයට, එරෙහිව, සටන්, ක...","[කැලිෆෝනියාව, ගොවිපල, ආශ්රිත, දුමාරය, සීමා, කි...","[දකුණු, කැලිෆෝනියාවේ, දුමාරයට, එරෙහිව, සටන්, ක..."


In [21]:
train_set['Stopwords Removed (Title)'] = train_set['Cleaned Title'].apply(lambda x: remove_stop_words(x))
train_set.head()

,Class Index,Title (Sinhala),Description (Sinhala),Cleaned Title,Cleaned Description,Stopwords Removed (Title)
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය.,"නිව් යෝර්ක් (රොයිටර්) - කෙටි විකුණුම්කරුවන්, ව...","[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,...","[නිව්, යෝර්ක්, රොයිටර්, කෙටි, විකුණුම්කරුවන්, ...","[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,..."
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්),රොයිටර් - පුද්ගලික ආයෝජන සමාගමක් වන Carlyle Gr...,"[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]","[රොයිටර්, පුද්ගලික, ආයෝජන, සමාගමක්, වන, ආරක්ෂක...","[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]"
2,3,තෙල් සහ ආර්ථිකය,රොයිටර් - බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබ...,"[තෙල්, සහ, ආර්ථිකය]","[රොයිටර්, බොරතෙල්, මිල, ඉහළ, යාම, සහ, ආර්ථිකය,...","[තෙල්, සහ, ආර්ථිකය]"
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,රොයිටර් - කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම්වල...,"[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,...","[රොයිටර්, කැරලිකාර, මිලීෂියාවකට, යටිතල, පහසුකම...","[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,..."
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"AFP - ඉරාක ලෝක තෙල් මිල, වාර්තා පෙරලීම සහ මුදල...","[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක...","[ඉරාක, ලෝක, තෙල්, මිල, වාර්තා, පෙරලීම, සහ, මුද...","[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක..."


In [22]:
train_set['Stopwords Removed (Description)'] = train_set['Cleaned Description'].apply(lambda x: remove_stop_words(x))
train_set.head()

,Class Index,Title (Sinhala),Description (Sinhala),Cleaned Title,Cleaned Description,Stopwords Removed (Title),Stopwords Removed (Description)
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය.,"නිව් යෝර්ක් (රොයිටර්) - කෙටි විකුණුම්කරුවන්, ව...","[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,...","[නිව්, යෝර්ක්, රොයිටර්, කෙටි, විකුණුම්කරුවන්, ...","[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,...","[නිව්, යෝර්ක්, රොයිටර්, කෙටි, විකුණුම්කරුවන්, ..."
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්),රොයිටර් - පුද්ගලික ආයෝජන සමාගමක් වන Carlyle Gr...,"[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]","[රොයිටර්, පුද්ගලික, ආයෝජන, සමාගමක්, වන, ආරක්ෂක...","[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]","[රොයිටර්, පුද්ගලික, ආයෝජන, සමාගමක්, වන, ආරක්ෂක..."
2,3,තෙල් සහ ආර්ථිකය,රොයිටර් - බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබ...,"[තෙල්, සහ, ආර්ථිකය]","[රොයිටර්, බොරතෙල්, මිල, ඉහළ, යාම, සහ, ආර්ථිකය,...","[තෙල්, සහ, ආර්ථිකය]","[රොයිටර්, බොරතෙල්, මිල, ඉහළ, යාම, සහ, ආර්ථිකය,..."
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,රොයිටර් - කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම්වල...,"[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,...","[රොයිටර්, කැරලිකාර, මිලීෂියාවකට, යටිතල, පහසුකම...","[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,...","[රොයිටර්, කැරලිකාර, මිලීෂියාවකට, යටිතල, පහසුකම..."
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"AFP - ඉරාක ලෝක තෙල් මිල, වාර්තා පෙරලීම සහ මුදල...","[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක...","[ඉරාක, ලෝක, තෙල්, මිල, වාර්තා, පෙරලීම, සහ, මුද...","[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක...","[ඉරාක, ලෝක, තෙල්, මිල, වාර්තා, පෙරලීම, සහ, මුද..."


## Stemming

In [23]:
stem_dict = open('/kaggle/input/stem-dictionary/stem_dictionary.txt', "r", encoding='utf-8')
stem_dict = map(lambda s: s.strip(), stem_dict)

stem_dictionary = {}

for s in stem_dict:
    s = s.split("\t")
    stem_dictionary[s[0]] = s[1]

In [24]:
def stemming_words(tokens):
    for k, v in enumerate(tokens):
        tokens[k] = stem_dictionary.get(v, v)
    tokens = " ".join([c for c in tokens if c not in string.punctuation])
    return tokens

In [25]:
test_set['Stemming Words (Title)'] = test_set['Stopwords Removed (Title)'].apply(lambda x: stemming_words(x))
test_set.head()

,Class Index,Title (Sinhala),Description (Sinhala),Cleaned Title,Cleaned Description,Stopwords Removed (Title),Stopwords Removed (Description),Stemming Words (Title)
0,3,කතා කරලා TN විශ්‍රාම වැටුපට බයයි,Turner Newall හි කම්කරුවන් නියෝජනය කරන වෘත්තීය...,"[කතා, කරලා, විශ්රාම, වැටුපට, බයයි]","[හි, කම්කරුවන්, නියෝජනය, කරන, වෘත්තීය, සමිති, ...","[කතා, කරලා, විශ්රාම, වැටුප, බයයි]","[හි, කම්කරුවන්, නියෝජනය, කරන, වෘත්තීය, සමිති, ...",කතා කරලා විශ්රාම වැටුප බයයි
1,4,තරඟය ක්‍රියාත්මකයි: දෙවන පුද්ගලික කණ්ඩායම මානව...,මිලියනයක් වන අන්සාරි X ත්‍යාගය සඳහා තරඟ වදින ද...,"[තරඟය, ක්රියාත්මකයි, දෙවන, පුද්ගලික, කණ්ඩායම, ...","[මිලියනයක්, වන, අන්සාරි, ත්යාගය, සඳහා, තරඟ, වද...","[තරඟ, ක්රියාත්මකයි, දෙවන, පුද්ගලික, කණ්ඩාය, මා...","[මිලියනයක්, වන, අන්සාරි, ත්යාගය, සඳහා, තරඟ, වද...",තරඟ ක්රියාත්මකයි දෙවන පුද්ගලික කණ්ඩාය මානව අභ්...
2,4,K. සමාගම Peptides (AP) අධ්‍යයනය සඳහා ප්‍රදානයක...,ලුයිස්විල් විශ්ව විද්‍යාලයේ රසායන විද්‍යා පර්ය...,"[සමාගම, අධ්යයනය, සඳහා, ප්රදානයක්, දිනා, ගනී]","[ලුයිස්විල්, විශ්ව, විද්යාලයේ, රසායන, විද්යා, ...","[සමාගම, අධ්යයනය, සඳහා, ප්රදානයක්, දිනා, ගනී]","[ලුයිස්විල්, විශ්ව, විද්යාලයේ, රසායන, විද්යා, ...",සමාගම අධ්යයනය සඳහා ප්රදානයක් දිනා ගනී
3,4,පුරෝකථන ඒකකය ලැව්ගිනි (AP) පුරෝකථනය කිරීමට උපක...,"මයික් ෆිට්ස්පැට්‍රික් වර්ණවත් සිතියම්, රූප සහ ...","[පුරෝකථන, ඒකකය, ලැව්ගිනි, පුරෝකථනය, කිරීමට, උප...","[මයික්, ෆිට්ස්පැට්රික්, වර්ණවත්, සිතියම්, රූප,...","[පුරෝකථන, ඒකකය, ලැව්ගිනි, පුරෝකථනය, කිරීමට, උප...","[මයික්, ෆිට්ස්පැට්රික්, වර්ණවත්, සිතියම්, රූප,...",පුරෝකථන ඒකකය ලැව්ගිනි පුරෝකථනය කිරීමට උපකාරී වේ
4,4,කැලිෆෝනියාව. ගොවිපල ආශ්‍රිත දුමාරය (AP) සීමා ක...,දකුණු කැලිෆෝනියාවේ දුමාරයට එරෙහිව සටන් කිරීමේ ...,"[කැලිෆෝනියාව, ගොවිපල, ආශ්රිත, දුමාරය, සීමා, කි...","[දකුණු, කැලිෆෝනියාවේ, දුමාරයට, එරෙහිව, සටන්, ක...","[කැලිෆෝනියාව, ගොවිපල, ආශ්රිත, දුමාරය, සීම, කිර...","[දකුණු, කැලිෆෝනියාවේ, දුමාරයට, එරෙහිව, සටන්, ක...",කැලිෆෝනියාව ගොවිපල ආශ්රිත දුමාරය සීම කිරීමේ අරමුණ


In [26]:
test_set['Stemming Words (Description)'] = test_set['Stopwords Removed (Description)'].apply(lambda x: stemming_words(x))
test_set.head()

,Class Index,Title (Sinhala),Description (Sinhala),Cleaned Title,Cleaned Description,Stopwords Removed (Title),Stopwords Removed (Description),Stemming Words (Title),Stemming Words (Description)
0,3,කතා කරලා TN විශ්‍රාම වැටුපට බයයි,Turner Newall හි කම්කරුවන් නියෝජනය කරන වෘත්තීය...,"[කතා, කරලා, විශ්රාම, වැටුපට, බයයි]","[හි, කම්කරුවන්, නියෝජනය, කරන, වෘත්තීය, සමිති, ...","[කතා, කරලා, විශ්රාම, වැටුප, බයයි]","[හි, කම්කරුවන්, නියෝජන, කරන, වෘත්තීය, සමිති, ප...",කතා කරලා විශ්රාම වැටුප බයයි,හි කම්කරුවන් නියෝජන කරන වෘත්තීය සමිති පවසන්නේ ...
1,4,තරඟය ක්‍රියාත්මකයි: දෙවන පුද්ගලික කණ්ඩායම මානව...,මිලියනයක් වන අන්සාරි X ත්‍යාගය සඳහා තරඟ වදින ද...,"[තරඟය, ක්රියාත්මකයි, දෙවන, පුද්ගලික, කණ්ඩායම, ...","[මිලියනයක්, වන, අන්සාරි, ත්යාගය, සඳහා, තරඟ, වද...","[තරඟ, ක්රියාත්මකයි, දෙවන, පුද්ගලික, කණ්ඩාය, මා...","[මිලියනයක, වන, අන්සාරි, ත්යාග, සඳහා, තරඟ, වදින...",තරඟ ක්රියාත්මකයි දෙවන පුද්ගලික කණ්ඩාය මානව අභ්...,මිලියනයක වන අන්සාරි ත්යාග සඳහා තරඟ වදින දෙවන ර...
2,4,K. සමාගම Peptides (AP) අධ්‍යයනය සඳහා ප්‍රදානයක...,ලුයිස්විල් විශ්ව විද්‍යාලයේ රසායන විද්‍යා පර්ය...,"[සමාගම, අධ්යයනය, සඳහා, ප්රදානයක්, දිනා, ගනී]","[ලුයිස්විල්, විශ්ව, විද්යාලයේ, රසායන, විද්යා, ...","[සමාගම, අධ්යයනය, සඳහා, ප්රදානයක්, දිනා, ගනී]","[ලුයිස්විල්, විශ්ව, විද්යාලයේ, රසායන, විද්යා, ...",සමාගම අධ්යයනය සඳහා ප්රදානයක් දිනා ගනී,ලුයිස්විල් විශ්ව විද්යාලයේ රසායන විද්යා පර්යේෂ...
3,4,පුරෝකථන ඒකකය ලැව්ගිනි (AP) පුරෝකථනය කිරීමට උපක...,"මයික් ෆිට්ස්පැට්‍රික් වර්ණවත් සිතියම්, රූප සහ ...","[පුරෝකථන, ඒකකය, ලැව්ගිනි, පුරෝකථනය, කිරීමට, උප...","[මයික්, ෆිට්ස්පැට්රික්, වර්ණවත්, සිතියම්, රූප,...","[පුරෝකථන, ඒකකය, ලැව්ගිනි, පුරෝකථනය, කිරීමට, උප...","[මයික්, ෆිට්ස්පැට්රික්, වර්ණවත්, සිතියම්, රූප,...",පුරෝකථන ඒකකය ලැව්ගිනි පුරෝකථනය කිරීමට උපකාරී වේ,මයික් ෆිට්ස්පැට්රික් වර්ණවත් සිතියම් රූප සහ නි...
4,4,කැලිෆෝනියාව. ගොවිපල ආශ්‍රිත දුමාරය (AP) සීමා ක...,දකුණු කැලිෆෝනියාවේ දුමාරයට එරෙහිව සටන් කිරීමේ ...,"[කැලිෆෝනියාව, ගොවිපල, ආශ්රිත, දුමාරය, සීමා, කි...","[දකුණු, කැලිෆෝනියාවේ, දුමාරයට, එරෙහිව, සටන්, ක...","[කැලිෆෝනියාව, ගොවිපල, ආශ්රිත, දුමාරය, සීම, කිර...","[දකුණු, කැලිෆෝනියා, දුමාරයට, එරෙහි, සටන්, කිරී...",කැලිෆෝනියාව ගොවිපල ආශ්රිත දුමාරය සීම කිරීමේ අරමුණ,දකුණු කැලිෆෝනියා දුමාරයට එරෙහි සටන් කිරීමේ ඒජන...


In [27]:
train_set['Stemming Words (Title)'] = train_set['Stopwords Removed (Title)'].apply(lambda x: stemming_words(x))
train_set.head()

,Class Index,Title (Sinhala),Description (Sinhala),Cleaned Title,Cleaned Description,Stopwords Removed (Title),Stopwords Removed (Description),Stemming Words (Title)
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය.,"නිව් යෝර්ක් (රොයිටර්) - කෙටි විකුණුම්කරුවන්, ව...","[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,...","[නිව්, යෝර්ක්, රොයිටර්, කෙටි, විකුණුම්කරුවන්, ...","[වෝල්, වීදිය, නැවත, ධනාත්මක, ආර්ථිකයක්, ලබා, ග...","[නිව්, යෝර්ක්, රොයිටර්, කෙටි, විකුණුම්කරුවන්, ...",වෝල් වීදිය නැවත ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්),රොයිටර් - පුද්ගලික ආයෝජන සමාගමක් වන Carlyle Gr...,"[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]","[රොයිටර්, පුද්ගලික, ආයෝජන, සමාගමක්, වන, ආරක්ෂක...","[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]","[රොයිටර්, පුද්ගලික, ආයෝජන, සමාගමක්, වන, ආරක්ෂක...",කාර්ලයිල් වාණිජ අභ්යවකාශය දෙස බලයි රොයිටර්
2,3,තෙල් සහ ආර්ථිකය,රොයිටර් - බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබ...,"[තෙල්, සහ, ආර්ථිකය]","[රොයිටර්, බොරතෙල්, මිල, ඉහළ, යාම, සහ, ආර්ථිකය,...","[තෙල්, සහ, ආර්ථිකය]","[රොයිටර්, බොරතෙල්, මිල, ඉහළ, යාම, සහ, ආර්ථිකය,...",තෙල් සහ ආර්ථිකය
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,රොයිටර් - කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම්වල...,"[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,...","[රොයිටර්, කැරලිකාර, මිලීෂියාවකට, යටිතල, පහසුකම...","[ඉරාක, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්, ...","[රොයිටර්, කැරලිකාර, මිලීෂියාවකට, යටිතල, පහසුකම...",ඉරාක ප්රධාන දකුණු නල මාර්ගයෙන් රොයිටර් තෙල් අප...
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"AFP - ඉරාක ලෝක තෙල් මිල, වාර්තා පෙරලීම සහ මුදල...","[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක...","[ඉරාක, ලෝක, තෙල්, මිල, වාර්තා, පෙරලීම, සහ, මුද...","[එක්සත, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, කර...","[ඉරාක, ලෝක, තෙල්, මිල, වාර්තා, පෙරලීම, සහ, මුද...",එක්සත ජනපද ආර්ථිකයට නව තර්ජනයක් එල්ල කරමින් තෙ...


In [28]:
train_set['Stemming Words (Description)'] = train_set['Stopwords Removed (Description)'].apply(lambda x: stemming_words(x))
train_set.head()

,Class Index,Title (Sinhala),Description (Sinhala),Cleaned Title,Cleaned Description,Stopwords Removed (Title),Stopwords Removed (Description),Stemming Words (Title),Stemming Words (Description)
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය.,"නිව් යෝර්ක් (රොයිටර්) - කෙටි විකුණුම්කරුවන්, ව...","[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,...","[නිව්, යෝර්ක්, රොයිටර්, කෙටි, විකුණුම්කරුවන්, ...","[වෝල්, වීදිය, නැවත, ධනාත්මක, ආර්ථිකයක්, ලබා, ග...","[නිව්, යෝර්ක්, රොයිටර්, කෙටි, විකුණුම්කරුවන්, ...",වෝල් වීදිය නැවත ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය,නිව් යෝර්ක් රොයිටර් කෙටි විකුණුම්කරුවන් වෝල් ස...
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්),රොයිටර් - පුද්ගලික ආයෝජන සමාගමක් වන Carlyle Gr...,"[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]","[රොයිටර්, පුද්ගලික, ආයෝජන, සමාගමක්, වන, ආරක්ෂක...","[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]","[රොයිටර්, පුද්ගලික, ආයෝජන, සමාගමක්, වන, ආරක්ෂක...",කාර්ලයිල් වාණිජ අභ්යවකාශය දෙස බලයි රොයිටර්,රොයිටර් පුද්ගලික ආයෝජන සමාගමක් වන ආරක්ෂක කර්මා...
2,3,තෙල් සහ ආර්ථිකය,රොයිටර් - බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබ...,"[තෙල්, සහ, ආර්ථිකය]","[රොයිටර්, බොරතෙල්, මිල, ඉහළ, යාම, සහ, ආර්ථිකය,...","[තෙල්, සහ, ආර්ථිකය]","[රොයිටර්, බොරතෙල්, මිල, ඉහළ, යාම, සහ, ආර්ථිකය,...",තෙල් සහ ආර්ථිකය,රොයිටර් බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබඳ ...
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,රොයිටර් - කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම්වල...,"[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,...","[රොයිටර්, කැරලිකාර, මිලීෂියාවකට, යටිතල, පහසුකම...","[ඉරාක, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්, ...","[රොයිටර්, කැරලිකාර, මිලීෂියාවකට, යටිතල, පහසුකම...",ඉරාක ප්රධාන දකුණු නල මාර්ගයෙන් රොයිටර් තෙල් අප...,රොයිටර් කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම් පහර...
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"AFP - ඉරාක ලෝක තෙල් මිල, වාර්තා පෙරලීම සහ මුදල...","[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක...","[ඉරාක, ලෝක, තෙල්, මිල, වාර්තා, පෙරලීම, සහ, මුද...","[එක්සත, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, කර...","[ඉරාක, ලෝක, තෙල්, මිල, වාර්තා, පෙරලීම, සහ, මුද...",එක්සත ජනපද ආර්ථිකයට නව තර්ජනයක් එල්ල කරමින් තෙ...,ඉරාක ලෝක තෙල් මිල වාර්තා පෙරලීම සහ මුදල් පසුම්...


## Remove Duplicates

In [29]:
test_set.drop_duplicates(subset='Stemming Words (Title)', keep='first', inplace=True)

test_set.head()

,Class Index,Title (Sinhala),Description (Sinhala),Cleaned Title,Cleaned Description,Stopwords Removed (Title),Stopwords Removed (Description),Stemming Words (Title),Stemming Words (Description)
0,3,කතා කරලා TN විශ්‍රාම වැටුපට බයයි,Turner Newall හි කම්කරුවන් නියෝජනය කරන වෘත්තීය...,"[කතා, කරලා, විශ්රාම, වැටුපට, බයයි]","[හි, කම්කරුවන්, නියෝජනය, කරන, වෘත්තීය, සමිති, ...","[කතා, කරලා, විශ්රාම, වැටුප, බයයි]","[හි, කම්කරුවන්, නියෝජන, කරන, වෘත්තීය, සමිති, ප...",කතා කරලා විශ්රාම වැටුප බයයි,හි කම්කරුවන් නියෝජන කරන වෘත්තීය සමිති පවසන්නේ ...
1,4,තරඟය ක්‍රියාත්මකයි: දෙවන පුද්ගලික කණ්ඩායම මානව...,මිලියනයක් වන අන්සාරි X ත්‍යාගය සඳහා තරඟ වදින ද...,"[තරඟය, ක්රියාත්මකයි, දෙවන, පුද්ගලික, කණ්ඩායම, ...","[මිලියනයක්, වන, අන්සාරි, ත්යාගය, සඳහා, තරඟ, වද...","[තරඟ, ක්රියාත්මකයි, දෙවන, පුද්ගලික, කණ්ඩාය, මා...","[මිලියනයක, වන, අන්සාරි, ත්යාග, සඳහා, තරඟ, වදින...",තරඟ ක්රියාත්මකයි දෙවන පුද්ගලික කණ්ඩාය මානව අභ්...,මිලියනයක වන අන්සාරි ත්යාග සඳහා තරඟ වදින දෙවන ර...
2,4,K. සමාගම Peptides (AP) අධ්‍යයනය සඳහා ප්‍රදානයක...,ලුයිස්විල් විශ්ව විද්‍යාලයේ රසායන විද්‍යා පර්ය...,"[සමාගම, අධ්යයනය, සඳහා, ප්රදානයක්, දිනා, ගනී]","[ලුයිස්විල්, විශ්ව, විද්යාලයේ, රසායන, විද්යා, ...","[සමාගම, අධ්යයනය, සඳහා, ප්රදානයක්, දිනා, ගනී]","[ලුයිස්විල්, විශ්ව, විද්යාලයේ, රසායන, විද්යා, ...",සමාගම අධ්යයනය සඳහා ප්රදානයක් දිනා ගනී,ලුයිස්විල් විශ්ව විද්යාලයේ රසායන විද්යා පර්යේෂ...
3,4,පුරෝකථන ඒකකය ලැව්ගිනි (AP) පුරෝකථනය කිරීමට උපක...,"මයික් ෆිට්ස්පැට්‍රික් වර්ණවත් සිතියම්, රූප සහ ...","[පුරෝකථන, ඒකකය, ලැව්ගිනි, පුරෝකථනය, කිරීමට, උප...","[මයික්, ෆිට්ස්පැට්රික්, වර්ණවත්, සිතියම්, රූප,...","[පුරෝකථන, ඒකකය, ලැව්ගිනි, පුරෝකථනය, කිරීමට, උප...","[මයික්, ෆිට්ස්පැට්රික්, වර්ණවත්, සිතියම්, රූප,...",පුරෝකථන ඒකකය ලැව්ගිනි පුරෝකථනය කිරීමට උපකාරී වේ,මයික් ෆිට්ස්පැට්රික් වර්ණවත් සිතියම් රූප සහ නි...
4,4,කැලිෆෝනියාව. ගොවිපල ආශ්‍රිත දුමාරය (AP) සීමා ක...,දකුණු කැලිෆෝනියාවේ දුමාරයට එරෙහිව සටන් කිරීමේ ...,"[කැලිෆෝනියාව, ගොවිපල, ආශ්රිත, දුමාරය, සීමා, කි...","[දකුණු, කැලිෆෝනියාවේ, දුමාරයට, එරෙහිව, සටන්, ක...","[කැලිෆෝනියාව, ගොවිපල, ආශ්රිත, දුමාරය, සීම, කිර...","[දකුණු, කැලිෆෝනියා, දුමාරයට, එරෙහි, සටන්, කිරී...",කැලිෆෝනියාව ගොවිපල ආශ්රිත දුමාරය සීම කිරීමේ අරමුණ,දකුණු කැලිෆෝනියා දුමාරයට එරෙහි සටන් කිරීමේ ඒජන...


In [30]:
test_set.drop_duplicates(subset='Stemming Words (Description)', keep='first', inplace=True)

test_set.head()

,Class Index,Title (Sinhala),Description (Sinhala),Cleaned Title,Cleaned Description,Stopwords Removed (Title),Stopwords Removed (Description),Stemming Words (Title),Stemming Words (Description)
0,3,කතා කරලා TN විශ්‍රාම වැටුපට බයයි,Turner Newall හි කම්කරුවන් නියෝජනය කරන වෘත්තීය...,"[කතා, කරලා, විශ්රාම, වැටුපට, බයයි]","[හි, කම්කරුවන්, නියෝජනය, කරන, වෘත්තීය, සමිති, ...","[කතා, කරලා, විශ්රාම, වැටුප, බයයි]","[හි, කම්කරුවන්, නියෝජන, කරන, වෘත්තීය, සමිති, ප...",කතා කරලා විශ්රාම වැටුප බයයි,හි කම්කරුවන් නියෝජන කරන වෘත්තීය සමිති පවසන්නේ ...
1,4,තරඟය ක්‍රියාත්මකයි: දෙවන පුද්ගලික කණ්ඩායම මානව...,මිලියනයක් වන අන්සාරි X ත්‍යාගය සඳහා තරඟ වදින ද...,"[තරඟය, ක්රියාත්මකයි, දෙවන, පුද්ගලික, කණ්ඩායම, ...","[මිලියනයක්, වන, අන්සාරි, ත්යාගය, සඳහා, තරඟ, වද...","[තරඟ, ක්රියාත්මකයි, දෙවන, පුද්ගලික, කණ්ඩාය, මා...","[මිලියනයක, වන, අන්සාරි, ත්යාග, සඳහා, තරඟ, වදින...",තරඟ ක්රියාත්මකයි දෙවන පුද්ගලික කණ්ඩාය මානව අභ්...,මිලියනයක වන අන්සාරි ත්යාග සඳහා තරඟ වදින දෙවන ර...
2,4,K. සමාගම Peptides (AP) අධ්‍යයනය සඳහා ප්‍රදානයක...,ලුයිස්විල් විශ්ව විද්‍යාලයේ රසායන විද්‍යා පර්ය...,"[සමාගම, අධ්යයනය, සඳහා, ප්රදානයක්, දිනා, ගනී]","[ලුයිස්විල්, විශ්ව, විද්යාලයේ, රසායන, විද්යා, ...","[සමාගම, අධ්යයනය, සඳහා, ප්රදානයක්, දිනා, ගනී]","[ලුයිස්විල්, විශ්ව, විද්යාලයේ, රසායන, විද්යා, ...",සමාගම අධ්යයනය සඳහා ප්රදානයක් දිනා ගනී,ලුයිස්විල් විශ්ව විද්යාලයේ රසායන විද්යා පර්යේෂ...
3,4,පුරෝකථන ඒකකය ලැව්ගිනි (AP) පුරෝකථනය කිරීමට උපක...,"මයික් ෆිට්ස්පැට්‍රික් වර්ණවත් සිතියම්, රූප සහ ...","[පුරෝකථන, ඒකකය, ලැව්ගිනි, පුරෝකථනය, කිරීමට, උප...","[මයික්, ෆිට්ස්පැට්රික්, වර්ණවත්, සිතියම්, රූප,...","[පුරෝකථන, ඒකකය, ලැව්ගිනි, පුරෝකථනය, කිරීමට, උප...","[මයික්, ෆිට්ස්පැට්රික්, වර්ණවත්, සිතියම්, රූප,...",පුරෝකථන ඒකකය ලැව්ගිනි පුරෝකථනය කිරීමට උපකාරී වේ,මයික් ෆිට්ස්පැට්රික් වර්ණවත් සිතියම් රූප සහ නි...
4,4,කැලිෆෝනියාව. ගොවිපල ආශ්‍රිත දුමාරය (AP) සීමා ක...,දකුණු කැලිෆෝනියාවේ දුමාරයට එරෙහිව සටන් කිරීමේ ...,"[කැලිෆෝනියාව, ගොවිපල, ආශ්රිත, දුමාරය, සීමා, කි...","[දකුණු, කැලිෆෝනියාවේ, දුමාරයට, එරෙහිව, සටන්, ක...","[කැලිෆෝනියාව, ගොවිපල, ආශ්රිත, දුමාරය, සීම, කිර...","[දකුණු, කැලිෆෝනියා, දුමාරයට, එරෙහි, සටන්, කිරී...",කැලිෆෝනියාව ගොවිපල ආශ්රිත දුමාරය සීම කිරීමේ අරමුණ,දකුණු කැලිෆෝනියා දුමාරයට එරෙහි සටන් කිරීමේ ඒජන...


In [31]:
test_set.drop_duplicates(subset='Stemming Words (Title)', keep='first', inplace=True)

train_set.head()

,Class Index,Title (Sinhala),Description (Sinhala),Cleaned Title,Cleaned Description,Stopwords Removed (Title),Stopwords Removed (Description),Stemming Words (Title),Stemming Words (Description)
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය.,"නිව් යෝර්ක් (රොයිටර්) - කෙටි විකුණුම්කරුවන්, ව...","[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,...","[නිව්, යෝර්ක්, රොයිටර්, කෙටි, විකුණුම්කරුවන්, ...","[වෝල්, වීදිය, නැවත, ධනාත්මක, ආර්ථිකයක්, ලබා, ග...","[නිව්, යෝර්ක්, රොයිටර්, කෙටි, විකුණුම්කරුවන්, ...",වෝල් වීදිය නැවත ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය,නිව් යෝර්ක් රොයිටර් කෙටි විකුණුම්කරුවන් වෝල් ස...
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්),රොයිටර් - පුද්ගලික ආයෝජන සමාගමක් වන Carlyle Gr...,"[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]","[රොයිටර්, පුද්ගලික, ආයෝජන, සමාගමක්, වන, ආරක්ෂක...","[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]","[රොයිටර්, පුද්ගලික, ආයෝජන, සමාගමක්, වන, ආරක්ෂක...",කාර්ලයිල් වාණිජ අභ්යවකාශය දෙස බලයි රොයිටර්,රොයිටර් පුද්ගලික ආයෝජන සමාගමක් වන ආරක්ෂක කර්මා...
2,3,තෙල් සහ ආර්ථිකය,රොයිටර් - බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබ...,"[තෙල්, සහ, ආර්ථිකය]","[රොයිටර්, බොරතෙල්, මිල, ඉහළ, යාම, සහ, ආර්ථිකය,...","[තෙල්, සහ, ආර්ථිකය]","[රොයිටර්, බොරතෙල්, මිල, ඉහළ, යාම, සහ, ආර්ථිකය,...",තෙල් සහ ආර්ථිකය,රොයිටර් බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබඳ ...
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,රොයිටර් - කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම්වල...,"[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,...","[රොයිටර්, කැරලිකාර, මිලීෂියාවකට, යටිතල, පහසුකම...","[ඉරාක, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්, ...","[රොයිටර්, කැරලිකාර, මිලීෂියාවකට, යටිතල, පහසුකම...",ඉරාක ප්රධාන දකුණු නල මාර්ගයෙන් රොයිටර් තෙල් අප...,රොයිටර් කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම් පහර...
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"AFP - ඉරාක ලෝක තෙල් මිල, වාර්තා පෙරලීම සහ මුදල...","[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක...","[ඉරාක, ලෝක, තෙල්, මිල, වාර්තා, පෙරලීම, සහ, මුද...","[එක්සත, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, කර...","[ඉරාක, ලෝක, තෙල්, මිල, වාර්තා, පෙරලීම, සහ, මුද...",එක්සත ජනපද ආර්ථිකයට නව තර්ජනයක් එල්ල කරමින් තෙ...,ඉරාක ලෝක තෙල් මිල වාර්තා පෙරලීම සහ මුදල් පසුම්...


In [32]:
test_set.drop_duplicates(subset='Stemming Words (Description)', keep='first', inplace=True)

train_set.head()

,Class Index,Title (Sinhala),Description (Sinhala),Cleaned Title,Cleaned Description,Stopwords Removed (Title),Stopwords Removed (Description),Stemming Words (Title),Stemming Words (Description)
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය.,"නිව් යෝර්ක් (රොයිටර්) - කෙටි විකුණුම්කරුවන්, ව...","[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,...","[නිව්, යෝර්ක්, රොයිටර්, කෙටි, විකුණුම්කරුවන්, ...","[වෝල්, වීදිය, නැවත, ධනාත්මක, ආර්ථිකයක්, ලබා, ග...","[නිව්, යෝර්ක්, රොයිටර්, කෙටි, විකුණුම්කරුවන්, ...",වෝල් වීදිය නැවත ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය,නිව් යෝර්ක් රොයිටර් කෙටි විකුණුම්කරුවන් වෝල් ස...
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්),රොයිටර් - පුද්ගලික ආයෝජන සමාගමක් වන Carlyle Gr...,"[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]","[රොයිටර්, පුද්ගලික, ආයෝජන, සමාගමක්, වන, ආරක්ෂක...","[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]","[රොයිටර්, පුද්ගලික, ආයෝජන, සමාගමක්, වන, ආරක්ෂක...",කාර්ලයිල් වාණිජ අභ්යවකාශය දෙස බලයි රොයිටර්,රොයිටර් පුද්ගලික ආයෝජන සමාගමක් වන ආරක්ෂක කර්මා...
2,3,තෙල් සහ ආර්ථිකය,රොයිටර් - බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබ...,"[තෙල්, සහ, ආර්ථිකය]","[රොයිටර්, බොරතෙල්, මිල, ඉහළ, යාම, සහ, ආර්ථිකය,...","[තෙල්, සහ, ආර්ථිකය]","[රොයිටර්, බොරතෙල්, මිල, ඉහළ, යාම, සහ, ආර්ථිකය,...",තෙල් සහ ආර්ථිකය,රොයිටර් බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබඳ ...
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,රොයිටර් - කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම්වල...,"[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,...","[රොයිටර්, කැරලිකාර, මිලීෂියාවකට, යටිතල, පහසුකම...","[ඉරාක, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්, ...","[රොයිටර්, කැරලිකාර, මිලීෂියාවකට, යටිතල, පහසුකම...",ඉරාක ප්රධාන දකුණු නල මාර්ගයෙන් රොයිටර් තෙල් අප...,රොයිටර් කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම් පහර...
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"AFP - ඉරාක ලෝක තෙල් මිල, වාර්තා පෙරලීම සහ මුදල...","[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක...","[ඉරාක, ලෝක, තෙල්, මිල, වාර්තා, පෙරලීම, සහ, මුද...","[එක්සත, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, කර...","[ඉරාක, ලෝක, තෙල්, මිල, වාර්තා, පෙරලීම, සහ, මුද...",එක්සත ජනපද ආර්ථිකයට නව තර්ජනයක් එල්ල කරමින් තෙ...,ඉරාක ලෝක තෙල් මිල වාර්තා පෙරලීම සහ මුදල් පසුම්...


## Setting Columns

In [33]:
train_set.drop(train_set.columns[[1, 2, 3, 4, 5, 6]], axis=1, inplace=True)
train_set.head()

,Class Index,Stemming Words (Title),Stemming Words (Description)
0,3,වෝල් වීදිය නැවත ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය,නිව් යෝර්ක් රොයිටර් කෙටි විකුණුම්කරුවන් වෝල් ස...
1,3,කාර්ලයිල් වාණිජ අභ්යවකාශය දෙස බලයි රොයිටර්,රොයිටර් පුද්ගලික ආයෝජන සමාගමක් වන ආරක්ෂක කර්මා...
2,3,තෙල් සහ ආර්ථිකය,රොයිටර් බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබඳ ...
3,3,ඉරාක ප්රධාන දකුණු නල මාර්ගයෙන් රොයිටර් තෙල් අප...,රොයිටර් කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම් පහර...
4,3,එක්සත ජනපද ආර්ථිකයට නව තර්ජනයක් එල්ල කරමින් තෙ...,ඉරාක ලෝක තෙල් මිල වාර්තා පෙරලීම සහ මුදල් පසුම්...


In [34]:
test_set.drop(test_set.columns[[1, 2, 3, 4, 5, 6]], axis=1, inplace=True)
test_set.head()

,Class Index,Stemming Words (Title),Stemming Words (Description)
0,3,කතා කරලා විශ්රාම වැටුප බයයි,හි කම්කරුවන් නියෝජන කරන වෘත්තීය සමිති පවසන්නේ ...
1,4,තරඟ ක්රියාත්මකයි දෙවන පුද්ගලික කණ්ඩාය මානව අභ්...,මිලියනයක වන අන්සාරි ත්යාග සඳහා තරඟ වදින දෙවන ර...
2,4,සමාගම අධ්යයනය සඳහා ප්රදානයක් දිනා ගනී,ලුයිස්විල් විශ්ව විද්යාලයේ රසායන විද්යා පර්යේෂ...
3,4,පුරෝකථන ඒකකය ලැව්ගිනි පුරෝකථනය කිරීමට උපකාරී වේ,මයික් ෆිට්ස්පැට්රික් වර්ණවත් සිතියම් රූප සහ නි...
4,4,කැලිෆෝනියාව ගොවිපල ආශ්රිත දුමාරය සීම කිරීමේ අරමුණ,දකුණු කැලිෆෝනියා දුමාරයට එරෙහි සටන් කිරීමේ ඒජන...


In [35]:
train_set.columns = ['ClassIndex', 'Stemming Words (Title)', 'Stemming Words (Description)']
test_set.columns = ['ClassIndex', 'Stemming Words (Title)', 'Stemming Words (Description)']

In [36]:
# X_train = train_set['Stemming Words (Title)'] + " " + train_set['Stemming Words (Description)']
# y_train = train_set['ClassIndex'].apply(lambda x: x-1).values

# X_test = test_set['Stemming Words (Title)'] + " " + test_set['Stemming Words (Description)'] 
# y_test = test_set['ClassIndex'].apply(lambda x: x-1).values

# maxlen = X_train.map(lambda x: len(x.split())).max()

In [37]:
train_set['Stemming Words'] = train_set['Stemming Words (Title)'] + " " + train_set['Stemming Words (Description)']

test_set['Stemming Words'] = test_set['Stemming Words (Title)'] + " " + test_set['Stemming Words (Description)']

In [38]:
train_set.head()

,ClassIndex,Stemming Words (Title),Stemming Words (Description),Stemming Words
0,3,වෝල් වීදිය නැවත ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය,නිව් යෝර්ක් රොයිටර් කෙටි විකුණුම්කරුවන් වෝල් ස...,වෝල් වීදිය නැවත ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය න...
1,3,කාර්ලයිල් වාණිජ අභ්යවකාශය දෙස බලයි රොයිටර්,රොයිටර් පුද්ගලික ආයෝජන සමාගමක් වන ආරක්ෂක කර්මා...,කාර්ලයිල් වාණිජ අභ්යවකාශය දෙස බලයි රොයිටර් රොය...
2,3,තෙල් සහ ආර්ථිකය,රොයිටර් බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබඳ ...,තෙල් සහ ආර්ථිකය රොයිටර් බොරතෙල් මිල ඉහළ යාම සහ...
3,3,ඉරාක ප්රධාන දකුණු නල මාර්ගයෙන් රොයිටර් තෙල් අප...,රොයිටර් කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම් පහර...,ඉරාක ප්රධාන දකුණු නල මාර්ගයෙන් රොයිටර් තෙල් අප...
4,3,එක්සත ජනපද ආර්ථිකයට නව තර්ජනයක් එල්ල කරමින් තෙ...,ඉරාක ලෝක තෙල් මිල වාර්තා පෙරලීම සහ මුදල් පසුම්...,එක්සත ජනපද ආර්ථිකයට නව තර්ජනයක් එල්ල කරමින් තෙ...


In [39]:
test_set.head()

,ClassIndex,Stemming Words (Title),Stemming Words (Description),Stemming Words
0,3,කතා කරලා විශ්රාම වැටුප බයයි,හි කම්කරුවන් නියෝජන කරන වෘත්තීය සමිති පවසන්නේ ...,කතා කරලා විශ්රාම වැටුප බයයි හි කම්කරුවන් නියෝජ...
1,4,තරඟ ක්රියාත්මකයි දෙවන පුද්ගලික කණ්ඩාය මානව අභ්...,මිලියනයක වන අන්සාරි ත්යාග සඳහා තරඟ වදින දෙවන ර...,තරඟ ක්රියාත්මකයි දෙවන පුද්ගලික කණ්ඩාය මානව අභ්...
2,4,සමාගම අධ්යයනය සඳහා ප්රදානයක් දිනා ගනී,ලුයිස්විල් විශ්ව විද්යාලයේ රසායන විද්යා පර්යේෂ...,සමාගම අධ්යයනය සඳහා ප්රදානයක් දිනා ගනී ලුයිස්වි...
3,4,පුරෝකථන ඒකකය ලැව්ගිනි පුරෝකථනය කිරීමට උපකාරී වේ,මයික් ෆිට්ස්පැට්රික් වර්ණවත් සිතියම් රූප සහ නි...,පුරෝකථන ඒකකය ලැව්ගිනි පුරෝකථනය කිරීමට උපකාරී ව...
4,4,කැලිෆෝනියාව ගොවිපල ආශ්රිත දුමාරය සීම කිරීමේ අරමුණ,දකුණු කැලිෆෝනියා දුමාරයට එරෙහි සටන් කිරීමේ ඒජන...,කැලිෆෝනියාව ගොවිපල ආශ්රිත දුමාරය සීම කිරීමේ අර...


In [40]:
test_set.drop(test_set.columns[[1, 2]], axis=1, inplace=True)
test_set.head()

,ClassIndex,Stemming Words
0,3,කතා කරලා විශ්රාම වැටුප බයයි හි කම්කරුවන් නියෝජ...
1,4,තරඟ ක්රියාත්මකයි දෙවන පුද්ගලික කණ්ඩාය මානව අභ්...
2,4,සමාගම අධ්යයනය සඳහා ප්රදානයක් දිනා ගනී ලුයිස්වි...
3,4,පුරෝකථන ඒකකය ලැව්ගිනි පුරෝකථනය කිරීමට උපකාරී ව...
4,4,කැලිෆෝනියාව ගොවිපල ආශ්රිත දුමාරය සීම කිරීමේ අර...


In [41]:
train_set.drop(train_set.columns[[1, 2]], axis=1, inplace=True)
train_set.head()

,ClassIndex,Stemming Words
0,3,වෝල් වීදිය නැවත ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය න...
1,3,කාර්ලයිල් වාණිජ අභ්යවකාශය දෙස බලයි රොයිටර් රොය...
2,3,තෙල් සහ ආර්ථිකය රොයිටර් බොරතෙල් මිල ඉහළ යාම සහ...
3,3,ඉරාක ප්රධාන දකුණු නල මාර්ගයෙන් රොයිටර් තෙල් අප...
4,3,එක්සත ජනපද ආර්ථිකයට නව තර්ජනයක් එල්ල කරමින් තෙ...


## Checking Missing Values

In [42]:
train_set.isnull().sum()

ClassIndex        0
Stemming Words    0
dtype: int64

In [43]:
test_set.isnull().sum()

ClassIndex        0
Stemming Words    0
dtype: int64

## Training

In [44]:
vectorizer = CountVectorizer()
X_test_vectorized = vectorizer.fit_transform(test_set['Stemming Words'])
X_train_vectorized = vectorizer.fit_transform(train_set['Stemming Words'])

In [45]:
y_test = test_set['ClassIndex']
y_train = train_set['ClassIndex']

In [49]:
model = MultinomialNB()
model.fit(X_train_vectorized, y_train)

MultinomialNB()

In [55]:
review = 'මෙවර විස්සයි විස්ස ලෝක කුසලාන කාන්තා ක්‍රිකට් තරගාවලියට සහභාගී වන ශ්‍රි ලංකා කාන්තා ක්‍රිකට් කණ්ඩායම අද අලුයම දකුණු අප්‍රිකාව බලා පිටත්ව ගොස් තිබේ.'
commenttype = model.predict(vectorizer.transform([review]))[0]
if commenttype =='0':
    print('World Comment')
elif commenttype =='1':
     print('Sports Comment')
elif commenttype =='2':
     print('Business & Economics Comment')
elif commenttype =='3':
     print('Science & Technology Comment')
else:
    print('Other Comment')

Other Comment
